Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A2 - Wikipedia, ORES, and Bias in Data
Please follow the reproducability workflow as practiced during the last exercise.

## Step 1⃣ | Data acquisition

You will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

In [1]:
import pandas as pd

wiki_articles = pd.read_csv("page_data.csv", sep = ",")

pop_data = pd.read_csv("export_2019.csv", sep = ";")

pop_data

,country,population,region
0,Algeria,44.357,AFRICA
1,Egypt,100.803,AFRICA
2,Libya,6.891,AFRICA
3,Morocco,35.952,AFRICA
4,Sudan,43.849,AFRICA
...,...,...,...
205,Samoa,0.200,OCEANIA
206,Solomon Islands,715.000,OCEANIA
207,Tonga,99.000,OCEANIA
208,Tuvalu,0.010,OCEANIA


## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that you will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

In [2]:
wiki_articles = wiki_articles[~wiki_articles["page"].astype(str).str.startswith("Template:")]
wiki_articles

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


### Getting article quality predictions with ORES

Now you need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For this assignment, you only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. You need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API we used for the last assignment. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

You will notice that ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. For this assignment, you only need to capture and use the value for prediction.

**❗Note:** It's possible that you will be unable to get a score for a particular article. If that happens, make sure to maintain a log of articles for which you were not able to retrieve an ORES score. This log should be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

You can use the following **samle code for API calls**:

In [3]:
import requests
import json

# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/lukaskaibel',
    'From': 'lukas.kaibel@fu-berlin.de'
}

def get_ores_data(rev_id, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : rev_id
              }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.dumps(response)

    return data

#returns 50 article ids, concatenated to a string, seperated by "|"
def get_rev_id_string(articles, i):
    rev_id_string = ""
    for rev_id in articles["rev_id"][(i*50):((i+1)*50)]:
        if rev_id_string == "":
            rev_id_string += str(rev_id)
        else:
            rev_id_string += "|" + str(rev_id)
    return rev_id_string

#returns tupel of predictions and no-predictions for 50 rev_ids (50 was limit for ores request)
def get_50_ores_predictions(rev_ids):
    ores_data_json = get_ores_data(rev_ids, headers)   #doesnt work for len(rev_ids) > 50 
    ores_data = pd.read_json(ores_data_json)
    scores = ores_data["enwiki"]["scores"]
    
    prediction_list = []
    no_prediction_list = []
    for rev_id in list(scores.keys()):
        wp10 = scores[str(rev_id)]["wp10"]
        if "score" in wp10:
            prediction_list.append([int(rev_id), wp10["score"]["prediction"]])
        else:
            no_prediction_list.append(int(rev_id))
    return (prediction_list, no_prediction_list)

#returns tupel of predictions and no-predictions for all articles by splitting them into groups of 50
def get_all_predictions(articles):
    predictions = []
    no_predictions = []
    iteration_length = len(wiki_articles["rev_id"])//50
    for i in range(iteration_length):
        print("\rPrediction download percentage: ", (100*i/iteration_length), end = "")
        rev_id_string = get_rev_id_string(wiki_articles, i)
        ores_predictions = get_50_ores_predictions(rev_id_string)
        predictions += ores_predictions[0]
        no_predictions += ores_predictions[1]
    print("\rPrediction download percentage: ", 100.000000000000000, end = "")
    print()
    return (predictions, no_predictions)

all_predictions = get_all_predictions(wiki_articles)
prediction_df = pd.DataFrame(all_predictions[0], columns = ["rev_id", "article_quality"])
prediction_df = pd.merge(prediction_df, wiki_articles, on = ["rev_id"], how = "inner")
prediction_df.to_csv("ORES_scores.csv")
print("Predictions saved as: 'ORES_scores.csv'")

no_prediction_df = pd.DataFrame(all_predictions[1], columns = ["rev_id"])
no_prediction_df = pd.merge(no_prediction_df, wiki_articles, on = ["rev_id"], how = "inner")
no_prediction_df.to_csv("ORES_no_scores.csv")
print("No_predictions saved as: 'ORES_no_scores.csv'")

Prediction download percentage:  3.64025695931477533

KeyboardInterrupt: 

Sending one request for each `rev_id` might take some time. If you want to send batches you can use `'|'.join(str(x) for x in revision_ids` to put your ids together. Please make sure to deal with [exception handling](https://www.w3schools.com/python/python_try_except.asp) of the `KeyError` exception, when extracting the `prediction` from the `JSON` response.

### Combining the datasets

Now you need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, you'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

Please remove any rows that do not have matching data, and output them to a `CSV` file called `countries-no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file should look like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

In [55]:
#merges the predictions and pop_data on "country"
predictions = pd.read_csv("ORES_scores.csv")
politicians_by_country = pd.merge(predictions, pop_data, on = ["country"], how = "inner")
del(politicians_by_country["Unnamed: 0"])
politicians_by_country.to_csv("politicians_by_country.csv")


## Step 3⃣ | Analysis

Your analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`)for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

In [134]:
pbc = pd.read_csv("politicians_by_country.csv")
del(pbc["Unnamed: 0"])
pbc["FA"] = 0
pbc["GA"] = 0
pbc["B"] = 0
pbc["C"] = 0
pbc["Start"] = 0
pbc["Stub"] = 0

#setting value for article quality --> makes it easier to group all similar countries, by just summing qualities up
for index, row in pbc.iterrows():
    art_qual = row["article_quality"]
    if art_qual == "FA":
        pbc.at[index, "FA"] = 1
    elif art_qual == "GA":
        pbc.at[index, "GA"] = 1
    elif art_qual == "B":
        pbc.at[index, "B"] = 1
    elif art_qual == "C":
        pbc.at[index, "C"] = 1
    elif art_qual == "Start":
        pbc.at[index, "Start"] = 1
    else:
        pbc.at[index, "Stub"] = 1

#summing up qualities and overall article count
pbc = pbc.groupby(["country", "population", "region"], as_index = False).agg({"FA" : "sum", "GA" : "sum", "B" : "sum", "C" : "sum", "Start" : "sum", "Stub" : "sum"})
pbc["article_count"] = pbc["FA"] + pbc["GA"] + pbc["B"] + pbc["C"] + pbc["Start"] + pbc["Stub"]
pbc


,country,population,region,FA,GA,B,C,Start,Stub,article_count
0,Afghanistan,38.928,ASIA,1,12,8,46,99,154,320
1,Albania,2.838,EUROPE,0,3,3,59,147,245,457
2,Algeria,44.357,AFRICA,0,2,3,10,44,57,116
3,Andorra,82.000,EUROPE,0,0,0,2,8,24,34
4,Angola,32.522,AFRICA,0,0,2,6,23,75,106
...,...,...,...,...,...,...,...,...,...,...
178,Venezuela,28.645,LATIN AMERICA AND THE CARIBBEAN,0,3,2,16,47,63,131
179,Vietnam,96.209,ASIA,7,6,6,50,55,63,187
180,Yemen,29.826,ASIA,1,2,1,12,32,69,117
181,Zambia,18.384,AFRICA,0,0,0,5,15,5,25


In [146]:
#calculate all the needed statistics "countries by coverage", ...
pbc["coverage"] = pbc["article_count"] / pbc["population"]
pbc["relative_quality"] = (pbc["FA"] + pbc["GA"]) / pbc["population"]
pbr = pbc.groupby(["region"], as_index = False).agg({"article_count" : "sum", "population" : "sum", "FA" : "sum", "GA" : "sum", "B" : "sum", "C" : "sum", "Start" : "sum", "Stub" : "sum"})
pbr["coverage"] = pbr["article_count"] / pbr["population"]
pbr["relative_quality"] = (pbr["FA"] + pbr["GA"]) / pbr["population"]

### Results format

The results from this analysis are six `data tables`. Embed these tables in the Jupyter notebook. You do not need to graph or otherwise visualize the data for this assignment. The tables will show:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

**❗Hint:** You will find what country belongs to which region (e.g. `ASIA`) also in `export_2019.csv`. You need to calculate the total poulation per region. For that you could use `groupby` and also check out `apply`.

In [128]:
pbc.sort_values(by = "coverage", inplace = True, ascending = False)
cvc_top = pbc.head(10)
cvc_top

,country,population,region,FA,GA,B,C,Start,Stub,article_count,coverage,relative_quality
169,Tuvalu,0.010,OCEANIA,0,4,0,6,34,10,54,5400.000000,400.000000
1,Albania,2.838,EUROPE,0,3,3,59,147,245,457,161.028894,1.057082
120,New Zealand,4.987,OCEANIA,2,11,2,91,234,443,783,157.008221,2.606778
124,Norway,5.387,EUROPE,3,6,0,36,152,459,656,121.774643,1.670689
109,Moldova,3.535,EUROPE,0,0,3,5,59,356,423,119.660537,0.000000
50,Estonia,1.331,EUROPE,0,0,1,9,47,91,148,111.194591,0.000000
54,Finland,5.529,EUROPE,0,0,2,22,115,430,569,102.911919,0.000000
139,Sao Tome and Principe,0.210,AFRICA,0,0,0,0,9,12,21,100.000000,0.000000
96,Lithuania,2.794,EUROPE,0,1,0,6,52,185,244,87.329993,0.357910
175,Uruguay,3.531,LATIN AMERICA AND THE CARIBBEAN,0,2,2,15,89,177,285,80.713679,0.566412


In [120]:
pbc.sort_values(by = "coverage", inplace = True, ascending = False)
cvc_bottom = pbc.tail(10)
cvc_bottom

,country,population,FA,GA,B,C,Start,Stub,article_count,coverage
112,Montenegro,622.0,0,2,4,10,25,31,72,0.115756
105,Martinique,356.0,0,1,1,0,3,29,34,0.095506
56,French Guiana,294.0,0,0,0,2,5,20,27,0.091837
155,Suriname,605.0,0,1,0,5,19,15,40,0.066116
30,Cape Verde,556.0,0,0,1,3,21,11,36,0.064748
11,Bahamas,393.0,0,0,0,4,9,7,20,0.050891
14,Barbados,287.0,0,0,0,5,3,6,14,0.048780
17,Belize,419.0,0,0,0,4,8,4,16,0.038186
43,Djibouti,988.0,0,0,1,9,15,12,37,0.037449
67,Guyana,787.0,0,0,1,5,8,6,20,0.025413


In [124]:
pbc.sort_values(by = "relative_quality", inplace = True, ascending = False)
crq_head = pbc.head(10)
crq_head

,country,population,FA,GA,B,C,Start,Stub,article_count,coverage,relative_coverage,relative_quality
169,Tuvalu,0.010,0,4,0,6,34,10,54,5400.000000,400.000000,400.000000
75,Ireland,5.003,8,17,24,116,137,71,373,74.555267,4.997002,4.997002
19,Bhutan,0.730,0,2,0,3,8,20,33,45.205479,2.739726,2.739726
120,New Zealand,4.987,2,11,2,91,234,443,783,157.008221,2.606778,2.606778
135,Romania,19.241,28,14,23,68,82,128,343,17.826516,2.182839,2.182839
76,Israel,9.216,6,14,8,103,259,103,493,53.493924,2.170139,2.170139
21,Bosnia-Herzegovina,3.281,1,5,2,27,66,69,170,51.813472,1.828711,1.828711
57,Gabon,2.231,2,2,4,19,32,38,97,43.478261,1.792918,1.792918
7,Armenia,2.956,2,3,0,26,59,104,194,65.629229,1.691475,1.691475
124,Norway,5.387,3,6,0,36,152,459,656,121.774643,1.670689,1.670689


In [125]:
pbc.sort_values(by = "relative_quality", inplace = True, ascending = False)
crq_tail = pbc.tail(10)
crq_tail

,country,population,FA,GA,B,C,Start,Stub,article_count,coverage,relative_coverage,relative_quality
11,Bahamas,393.000,0,0,0,4,9,7,20,0.050891,0.0,0.0
14,Barbados,287.000,0,0,0,5,3,6,14,0.048780,0.0,0.0
17,Belize,419.000,0,0,0,4,8,4,16,0.038186,0.0,0.0
5,Antigua and Barbuda,98.000,0,0,0,8,9,7,24,0.244898,0.0,0.0
43,Djibouti,988.000,0,0,1,9,15,12,37,0.037449,0.0,0.0
62,Grenada,113.000,0,0,0,4,9,23,36,0.318584,0.0,0.0
83,Kiribati,125.000,0,0,0,3,6,21,30,0.240000,0.0,0.0
166,Tunisia,11.896,0,0,2,10,43,83,138,11.600538,0.0,0.0
81,Kazakhstan,18.732,0,0,2,15,23,38,78,4.163997,0.0,0.0
67,Guyana,787.000,0,0,1,5,8,6,20,0.025413,0.0,0.0


In [143]:
pbr.sort_values(by = "coverage", inplace = True, ascending = False)
rbc_head = pbr.head(10)
rbc_head

,region,article_count,population,FA,GA,B,C,Start,Stub,coverage
4,NORTHERN AMERICA,1910,368.068,31,73,69,461,759,517,5.189259
2,EUROPE,15776,3068.144,126,224,243,1766,4763,8654,5.141871
0,AFRICA,6844,2859.681,31,88,86,893,2222,3524,2.393274
1,ASIA,11691,5147.988,59,257,229,1625,3693,5828,2.270984
5,OCEANIA,3127,2369.701,14,49,19,378,976,1691,1.319576
3,LATIN AMERICA AND THE CARIBBEAN,5273,4425.712,20,56,50,542,1458,3147,1.191447


In [147]:
pbr.sort_values(by = "relative_quality", inplace = True, ascending = False)
rrq_head = pbr.head(10)
rrq_head

,region,article_count,population,FA,GA,B,C,Start,Stub,coverage,relative_quality
4,NORTHERN AMERICA,1910,368.068,31,73,69,461,759,517,5.189259,0.282556
2,EUROPE,15776,3068.144,126,224,243,1766,4763,8654,5.141871,0.114075
1,ASIA,11691,5147.988,59,257,229,1625,3693,5828,2.270984,0.061383
0,AFRICA,6844,2859.681,31,88,86,893,2222,3524,2.393274,0.041613
5,OCEANIA,3127,2369.701,14,49,19,378,976,1691,1.319576,0.026586
3,LATIN AMERICA AND THE CARIBBEAN,5273,4425.712,20,56,50,542,1458,3147,1.191447,0.017172


***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).